In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

## 4: Multiplying Matrices
- test that $\sigma_j\sigma_k = \delta_{jk}\mathbf{1} + i \epsilon_{jkl}\sigma_l$

In [21]:
#from 2024-1
def norm_diff(L, M):
    LminusM = L - M
    mag_LminusM = np.sqrt(np.sum(np.abs(LminusM)**2))
    mag_L = np.sqrt(np.sum(np.abs(L)**2))
    mag_M = np.sqrt(np.sum(np.abs(M)**2))
    
    delta = mag_LminusM / (mag_L + mag_M)
    
    return delta

## 5: Create function to multiply matrices
- (5a): MAKE GENERAL MATRIX MAKER: create function that inputs 2 arrays of complex numbers ($A_0, \vec{A}$) and ($B_0, \vec{B}$) and output array of complex numbers: ($C_0, \vec{C}$)
- (5b) create function that outputs array of 4 random complex numbers
- (5c) Test function using random A and B arrays compare two ways

In [118]:
##5a: make general matrix maker
def matrix_maker(m0xyz): #array to matrix
    M0, Mx, My, Mz = m0xyz
    
    identity_matrix = np.zeros((2,2), dtype = 'complex128')
    identity_matrix[0,0]= 1
    identity_matrix[1,1] = 1

    pauli_x = np.zeros((2,2), dtype = 'complex128')
    pauli_x[0,1]= 1
    pauli_x[1,0] = 1

    pauli_y = np.zeros((2,2), dtype = 'complex128')
    pauli_y[0,1]= -1j
    pauli_y[1,0] = 1j

    pauli_z = np.zeros((2,2), dtype = 'complex128')
    pauli_z[0,0]= 1
    pauli_z[1,1] = -1
    
    m = np.array([Mx, My, Mz])
    
    pauli_basis = np.array([pauli_x, pauli_y, pauli_z])
    
    m_dot_pauli = np.tensordot(m, pauli_basis, axes=1)
    
    matrix = M0 * identity_matrix + m_dot_pauli
    
    return matrix

In [119]:
a0xyz = [1, 1, 1, 1]
b0xyz = [2, 2, 2, 2]
A = matrix_maker(a0xyz)
B = matrix_maker(b0xyz)
C = A@B
print(A)
print(B)
print(C)

[[2.+0.j 1.-1.j]
 [1.+1.j 0.+0.j]]
[[4.+0.j 2.-2.j]
 [2.+2.j 0.+0.j]]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [5]:
## takes two arrays and creates a third that can then be used to make a matrix
def calculate_c0xyz(a0xyz, b0xyz): #arrays to array
    A0, Ax, Ay, Az = a0xyz
    B0, Bx, By, Bz = b0xyz
    axyz = np.array([Ax, Ay, Az])
    bxyz = np.array([Bx, By, Bz])
    C0 = A0*B0 + np.tensordot(axyz, bxyz, axes=1)
    cxyz = A0*bxyz + B0*axyz +1j*(np.cross(axyz, bxyz))
    c0xyz = ([C0, cxyz[0], cxyz[1], cxyz[2]])
    return c0xyz

In [12]:
#testing calculate_c0xyz
e0xyz = [1, 1, 1, 1]
f0xyz = [2, 2, 2, 2]
g0xyz = calculate_c0xyz(e0xyz, f0xyz)
print(g0xyz)
G = matrix_maker(g0xyz)
print(G)

[8.+0.j 4.+0.j 4.+0.j 4.+0.j]
[[12.+0.j  4.-4.j]
 [ 4.+4.j  4.+0.j]]


In [39]:
##5b make random number generator
    ## np.random.randn(n) returns a 1D array with n random numbers, so here it makes a 1D array of 4 random numbers (could make a matrix of random numbers or a 2D array i should say, if you give two inputs like np.random.randn(m,n))
    ## and "randn" refers to "random normal" which means it generates numbers from the standard normal distribution (gaussian distribution)
def random_complex_numbers():
    real_parts = np.random.randn(4)
    imaginary_parts = np.random.randn(4)
    complex_numbers = real_parts + 1j*imaginary_parts
    return complex_numbers

random_numbers = random_complex_numbers()
print(random_numbers)

[ 0.27876178+0.22784961j  0.41354682-0.44730252j  0.75574042-2.19655844j
 -1.27533843-1.41615479j]


In [14]:
#take a0xyz and b0xyz and make c0xyz then C
def one_matrix_test(a0xyz, b0xyz): #arrays to matrix
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C = matrix_maker(c0xyz)
    return C

#take a0xyz and b0xyz and make A and B then C
def three_matrix_test(a0xyz, b0xyz): #arrays to matrix
    A = matrix_maker(a0xyz)
    B = matrix_maker(b0xyz)
    C = A@B
    return C

In [16]:
#testing
h0xyz = [1, 1, 1, 1]
i0xyz = [2, 2, 2, 2]
C1 = one_matrix_test(h0xyz, i0xyz)
C2 = three_matrix_test(h0xyz, i0xyz)

In [120]:
norm_diff(C1, C2)
print(C1)
print (C2)

[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


In [121]:
### 5c: testing function using random numbers
def trial_loop(N): 

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()

        #compute density matrix both ways
        L = one_matrix_test(random_numbers1, random_numbers2)
        M = three_matrix_test(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [122]:
N = 100
delta_values, random_numbers_used = trial_loop(N) #run like this to capture outputs

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [5.168893833722828e-17, 5.248956009258442e-17, 8.281124774529506e-17, 6.229585687042398e-17, 2.6806645090351926e-17, 8.68963500706568e-17, 1.0157124324555338e-16, 9.461151435210369e-17, 2.031118350744297e-17, 8.188387984124253e-17, 7.962773762592149e-17, 7.730500846956042e-17, 1.0032280065483114e-16, 3.8612270349688547e-17, 6.607335748211443e-17, 6.2411999328141e-17, 7.829918363947219e-17, 1.0764510462596408e-16, 7.349217646565194e-17, 9.334930462923156e-17, 6.422236029083332e-17, 4.216763018052191e-17, 1.3052950854006576e-16, 3.505496418095723e-17, 3.8017034062021485e-17, 9.769268557246133e-17, 9.714541224734784e-17, 5.814127471599145e-17, 9.867104954429849e-17, 9.310162717758142e-17, 1.0172975852853489e-16, 4.5442665988426283e-17, 5.0956490595587176e-17, 6.001862950363992e-17, 6.476366087197112e-17, 6.879241490697155e-17, 6.475501832103152e-17, 8.533202170201055e-17, 7.07226210898209e-17, 7.57816801168664e-17, 7.30096065011715e-17, 4.610566347154471e-17, 8.7596220948360

## 6: Create matrix multiplication for $(\mathbf{1} - \rho^{(2)})\rho^{(4)}$
- (6a): create functions that input $(P_0, \vec{P})$ and output $(A_0, \vec{A})$ and $(B_0, \vec{B})$
- (6b): test

In [123]:
## 6a
def array_for_rho4(p0xyz): 
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    B0 = 0.5*P0
    bxyz = 0.5*P0*pxyz
    b0xyz = ([B0, bxyz[0], bxyz[1], bxyz[2]])
    return b0xyz #makes array to be put into matrix maker to create rho

def array_for_rho2(p0xyz):
    P0, Px, Py, Pz = p0xyz
    pxyz = np.array([Px, Py, Pz])
    A0 = 1 - 0.5*P0
    axyz = -0.5*P0*pxyz
    a0xyz = ([A0, axyz[0], axyz[1], axyz[2]])
    return a0xyz
#i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

^ i think the point of this is to take the specific array and matrix and make it general so I can put into general matrix maker. If i was using create_density_matrix, i could probably put p0xyz directly in and it would make the matrix?

In [124]:
## 6b testing
p0xyz = [1, 1, 1, 1]
b0xyz = array_for_rho4(p0xyz) #how to check this
print('ARRAY B:', b0xyz)
rho4 = matrix_maker(b0xyz)
print('RHO4:',rho4)

p0xyz = [1, 1, 1, 1]
a0xyz = array_for_rho2(p0xyz) #how to check this
print('ARRAY A:', a0xyz)
rho2 = matrix_maker(a0xyz)
print('RHO2:', rho2)

ARRAY B: [0.5, 0.5, 0.5, 0.5]
RHO4: [[1. +0.j  0.5-0.5j]
 [0.5+0.5j 0. +0.j ]]
ARRAY A: [0.5, -0.5, -0.5, -0.5]
RHO2: [[ 0. +0.j  -0.5+0.5j]
 [-0.5-0.5j  1. +0.j ]]


## 7: Calculate $ C = (\mathbf{1} - \rho^{(2)})\rho^{(4)} = C_0\mathbf{1} + \vec{C}* \vec{\sigma}$
- (7a): create function that inputs 2 arrays ($P_0^{(2)}, \vec{P^{(2)}}$) and ($P_0^{(4)}, \vec{P^{(4)}}$) and outputs $(C_0, \vec{C})$
- (7b): test

- $\textbf{create_rho2}$ creates an array that can be put into matrix maker to create matrix "rho2"
- $\textbf{create_rho4}$ creates an array that can be put into matrix maker to create matrix "rho4"

- "rho2" is a matrix:  $(\mathbf{1} - \rho^{(2)})$
- "rho4" is a matrix: $\rho^{(4)}$

- so to create rho2 or rho4, need to take p2 or p4 and input it into the correct function, then we will have an array to put into matrix maker to get rho2 and rho4?
..... i am getting lost

In [127]:
#### dont need this: do need this!!!
def rho2_matrix(p0xyz_2):
    rho2_array = array_for_rho2(p0xyz_2)
    rho2 = matrix_maker(rho2_array)
    return rho2

def rho4_matrix(p0xyz_4):
    rho4_array = array_for_rho4(p0xyz_4)
    rho4 = matrix_maker(rho4_array)
    return rho4

In [128]:
##7a
def rho2timesrho4(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    return c0xyz

p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
c10xyz = rho2timesrho4(p0xyz_2, p0xyz_4)
C1 = matrix_maker(c0xyz)
print(c10xyz)
print(C1)

[-0.5, 0j, 0j, 0j]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


COMPARING FUNCTIONS:
- function 1, $\textbf{final_C_1}$ is taking in p2 and p4 arrays, creating a0xyz and b0xyz arrays to then get a third array c0xyz which can go into matrix maker and get the final matrix C
- function 2, $\textbf{final_C_2}$ is taking in p2 and p4 arrays and making rho2 and rho4 matrices to then be multiplied and create matrix C


In [129]:
##7b: comparing functions:
def final_C_1(p0xyz_2, p0xyz_4):
    #make a0xyz and b0xyz from p0xyz_2 and p0xyz_4 -- thats what array_for_rho4 functions do!
    a0xyz = array_for_rho2(p0xyz_2)
    b0xyz = array_for_rho4(p0xyz_4)
    c0xyz = calculate_c0xyz(a0xyz, b0xyz)
    C1 = matrix_maker(c0xyz)
    return C1

def final_C_2(p0xyz_2, p0xyz_4):
    rho2 = rho2_matrix(p0xyz_2)
    rho4 = rho4_matrix(p0xyz_4)
    C2 = rho2@rho4
    return C2

In [130]:
p0xyz_2 = [1, 1, 1, 1]
p0xyz_4 = [1, 1, 1, 1]
C1 = final_C_1(p0xyz_2, p0xyz_4)
C2 = final_C_2(p0xyz_2, p0xyz_4)
print (C1)
print (C2)

[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]
[[-0.5+0.j  0. +0.j]
 [ 0. +0.j -0.5+0.j]]


In [131]:
### testing with random numbers
def trial_loop(N):

    delta_values = [] #an empty list so i can append later
    random_numbers_used = []

    for i in range(N):
        #create random array
        random_numbers1 = random_complex_numbers()
        random_numbers2 = random_complex_numbers()
        
        L = final_C_1(random_numbers1, random_numbers2)
        M = final_C_2(random_numbers1, random_numbers2)

        #compare the matrices
        delta = norm_diff(L, M)

        #save the delta values into an array and append the random arrays to a list
        delta_values.append(delta)
        random_numbers_used.append((random_numbers1, random_numbers2)) #called it something different because i was trying to append random numbers to itslef and it said no
    
    return delta_values, random_numbers_used

In [117]:
N = 100 
delta_values, random_numbers_used = trial_loop(N)

print("DELTA VALUES:", delta_values)
print("RANDOM NUMBERS USED:", random_numbers_used)

DELTA VALUES: [4.8554242978722504e-17, 6.97201213467952e-17, 7.060237687129724e-17, 9.59029126616905e-17, 4.503514156069371e-17, 9.752890221509919e-17, 7.192273552531138e-17, 2.949837771325019e-17, 8.916993108583792e-17, 1.010884277432363e-16, 5.575919984470112e-17, 8.899517788255936e-17, 9.355056206984296e-17, 6.540300251311456e-17, 7.522269377709949e-17, 8.003721868220582e-17, 6.799014816393222e-17, 5.076313798851311e-17, 8.52589596870144e-17, 6.499121561516093e-17, 1.006571213160125e-16, 8.954094887995454e-17, 8.669314934382656e-17, 7.647686861474621e-17, 4.1900901562534616e-17, 1.5335780144866298e-16, 8.342835049290669e-17, 1.478562389812592e-16, 5.905634506360749e-17, 6.671749207416748e-17, 5.825558421073188e-17, 8.895919478833532e-17, 3.71591254893737e-17, 7.252287391540736e-17, 8.487582274791281e-17, 1.116029610094598e-16, 3.3205131512228744e-17, 1.4762268259368382e-16, 7.043889742998678e-17, 9.575859559616691e-17, 5.695443581862265e-17, 1.5538747655046768e-16, 6.353757364485666